# Movie Recommendation System

## Goal
Build and evaluate a top-K movie recommender system using explicit user ratings.

The task is formulated as a ranking problem, not rating prediction.

This project demonstrates:
- recommender system fundamentals
- offline evaluation methodology
- baseline comparison
- reproducible ML workflow


## Imports

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from typing import Dict, List
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import numpy as np



## Dataset loading

In [4]:
ratings = pd.read_csv(
    "https://raw.githubusercontent.com/aiedu-courses/stepik_applied_tasks/main/datasets/movies_ratings.csv"
)

ratings.head()

,userId,movieId,rating,timestamp,title
0,1,31,2.5,1260759144,Dangerous Minds
1,7,31,3.0,851868750,Dangerous Minds
2,31,31,4.0,1273541953,Dangerous Minds
3,32,31,4.0,834828440,Dangerous Minds
4,36,31,3.0,847057202,Dangerous Minds


# Dataset Description

Columns:
- userId - user identifier
- movieId - movie identifier
- rating - explicit rating (0.5–5.0)
- timestamp - interaction time
- title - movie title

Each row represents a user–movie interaction.

In [131]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings["userId"] = user_encoder.fit_transform(ratings["userId"])
ratings["movieId"] = item_encoder.fit_transform(ratings["movieId"])

num_users = ratings["userId"].nunique()
num_items = ratings["movieId"].nunique()

num_users, num_items


(671, 9025)

In [132]:
item_id_to_title = (
    ratings[["movieId", "title"]]
    .drop_duplicates()
    .set_index("movieId")["title"]
    .to_dict()
)

## Problem formulation
-> For each user, recommend K unseen movies


## Data splitting

In [6]:
# Train/test split (time-aware context)
NUM_TEST_ITEMS = 10

train_parts = []
test_parts = []

for user_id, user_data in ratings.groupby("userId"):
    user_data = user_data.sort_values("timestamp")
    train_parts.append(user_data[:-NUM_TEST_ITEMS])
    test_parts.append(user_data[-NUM_TEST_ITEMS:])

train = pd.concat(train_parts)
test = pd.concat(test_parts)

train.shape, test.shape


((93140, 5), (6710, 5))

In [7]:
train.head()

,userId,movieId,rating,timestamp,title
706,0,1811,2.0,1260759108,Antz
759,0,1958,2.5,1260759113,The Fly
849,0,2920,3.0,1260759117,Blazing Saddles
351,0,1079,3.5,1260759125,Dracula
403,0,1083,2.0,1260759131,Cape Fear


In [8]:
test.head()

,userId,movieId,rating,timestamp,title
42,0,830,3.0,1260759179,Dumbo
84,0,856,3.0,1260759182,Sleepers
117,0,903,2.0,1260759185,Escape from New York
259,0,1037,2.0,1260759187,Ben-Hur
535,0,1511,4.0,1260759191,The French Connection


## Getting the right answers

In [9]:
true_items = test.groupby("userId")["movieId"].apply(list).to_dict()


In [10]:
true_items[0]

[830, 856, 903, 1037, 1511, 1704, 1739, 2375, 1136, 927]

## Metrics

In [11]:
def recall_at_k(recommended, relevant, k) -> float:
    recommended_k = set(recommended[:k])
    relevant_set = set(relevant)

    if len(relevant_set) == 0:
        return 0.0

    return len(recommended_k & relevant_set) / len(relevant_set)
import numpy as np

def ndcg_at_k(recommended, relevant, k: int) -> float:
    recommended = recommended[:k]
    relevant_set = set(relevant)

    if len(relevant_set) == 0:
        return 0.0

    dcg = 0.0
    for idx, item in enumerate(recommended):
        if item in relevant_set:
            dcg += 1.0 / np.log2(idx + 2)

    ideal_hits = min(len(relevant_set), k)
    idcg = sum(1.0 / np.log2(i + 2) for i in range(ideal_hits))

    return dcg / idcg
import numpy as np

def map_at_k(recommended, relevant, k: int) -> float:
    recommended = recommended[:k]
    relevant_set = set(relevant)

    if len(relevant_set) == 0:
        return 0.0

    score = 0.0
    hits = 0

    for idx, item in enumerate(recommended):
        if item in relevant_set:
            hits += 1
            score += hits / (idx + 1)

    return score / min(len(relevant_set), k)




In [12]:
def evaluate_model_extended(recommend_fn, true_items, k: int = 10):
    recalls, ndcgs, maps = [], [], []

    for user_id, relevant_items in true_items.items():
        rec_items = recommend_fn(user_id)

        recalls.append(recall_at_k(rec_items, relevant_items, k))
        ndcgs.append(ndcg_at_k(rec_items, relevant_items, k))
        maps.append(map_at_k(rec_items, relevant_items, k))

    return {
        f"Recall@{k}": float(np.mean(recalls)),
        f"NDCG@{k}": float(np.mean(ndcgs)),
        f"MAP@{k}": float(np.mean(maps)),
    }


In [13]:
recall_at_k([1, 3, 5], [3, 7, 8], 3)

0.3333333333333333

# Model 1 — Baseline

**Idea** : The baseline recommender ranks movies by their average rating in the training set
and recommends the same Top-K movies to every user.



## Baseline Model (top popularity movies)

In [14]:
train.head()

,userId,movieId,rating,timestamp,title
706,0,1811,2.0,1260759108,Antz
759,0,1958,2.5,1260759113,The Fly
849,0,2920,3.0,1260759117,Blazing Saddles
351,0,1079,3.5,1260759125,Dracula
403,0,1083,2.0,1260759131,Cape Fear


In [15]:
popular_items = train.groupby('movieId').rating.mean().sort_values(ascending=False).to_list()

In [16]:
def recommend_top_popularity(k):
    return popular_items[:k]

## Baseline evaluation

In [17]:
baseline_scores = evaluate_model_extended(
    recommend_top_popularity,
    true_items,
    k=10
)

baseline_scores

{'Recall@10': 0.0004470938897168406,
 'NDCG@10': 0.004470938897168405,
 'MAP@10': 0.004470938897168405}

# Model 2 — User-based Collaborative Filtering (UBCF)

**Idea:** recommend items to a user based on preferences of *similar users*.

In [18]:
RATING_THRESHOLD = 4.0
K_NEIGHBORS = 50
TOP_N = 10

In [19]:
from scipy.sparse import csr_matrix

n_users = ratings["userId"].nunique()
n_items = ratings["movieId"].nunique()

rows = train["userId"].to_numpy()
cols = train["movieId"].to_numpy()
vals = train["rating"].to_numpy().astype(np.float32)

R_train = csr_matrix((vals, (rows, cols)), shape=(n_users, n_items))

In [20]:
user_rating_counts = np.asarray((R_train != 0).sum(axis=1)).ravel()
user_rating_sums = np.asarray(R_train.sum(axis=1)).ravel()

user_means = np.zeros(n_users, dtype=np.float32)
mask = user_rating_counts > 0
user_means[mask] = user_rating_sums[mask] / user_rating_counts[mask]


In [21]:
R_centered = R_train.copy().tocsr()

indptr = R_centered.indptr
data = R_centered.data

for u in range(n_users):
    start, end = indptr[u], indptr[u + 1]
    if start != end:
        data[start:end] -= user_means[u]


In [22]:
from sklearn.neighbors import NearestNeighbors

nn_model = NearestNeighbors(
    n_neighbors=K_NEIGHBORS + 1,
    metric="cosine",
    algorithm="brute",
    n_jobs=-1
)

nn_model.fit(R_centered)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=51)

In [23]:
def recommend_ubcf(user_id, top_n = 10, k_neighbors = 50):
    user_profile = R_train[user_id]
    seen_items = set(user_profile.indices.tolist())

    distances, neighbors = nn_model.kneighbors(R_centered[user_id], n_neighbors=k_neighbors + 1)
    distances = distances.ravel()
    neighbors = neighbors.ravel()

    if neighbors[0] == user_id:
        neighbors = neighbors[1:]
        distances = distances[1:]

    sims = 1.0 - distances
    sims = np.clip(sims, 0.0, 1.0)

    scores = np.zeros(n_items, dtype=np.float32)
    sim_sums = np.zeros(n_items, dtype=np.float32)

    for v, sim in zip(neighbors, sims):
        if sim <= 1e-8:
            continue

        v_row = R_train[v]
        v_items = v_row.indices
        v_ratings = v_row.data

        v_centered = v_ratings - user_means[v]

        scores[v_items] += sim * v_centered
        sim_sums[v_items] += sim

    nonzero = sim_sums > 0
    scores[nonzero] = scores[nonzero] / sim_sums[nonzero]

    scores = scores + user_means[user_id]

    if seen_items:
        scores[list(seen_items)] = -np.inf

    top_items = np.argpartition(-scores, kth=min(top_n, n_items-1))[:top_n]
    top_items = top_items[np.argsort(-scores[top_items])]

    return [int(i) for i in top_items]


In [24]:
example_user = int(train["userId"].iloc[0])
recs = recommend_ubcf(example_user, top_n=TOP_N, k_neighbors=K_NEIGHBORS)
recs[:10]


[5056, 911, 4364, 5428, 3471, 4594, 8793, 3424, 979, 7602]

## Evaluation UBCF

In [25]:
ubcf_scores = evaluate_model_extended(
    recommend_ubcf,
    true_items,
    k=TOP_N
)

ubcf_scores

{'Recall@10': 0.001208809405530717,
 'NDCG@10': 0.0015680021031523738,
 'MAP@10': 0.0005644264660658103}

# Model 3 — Item-based Collaborative Filtering (IBCF)

**Idea** : A user is recommended movies that are similar to the movies they liked before.



In [26]:
K_ITEM_NEIGHBORS = 100
TOP_N = 10
RATING_THRESHOLD = 4.0
MIN_SIMILARITY = 0.0


In [27]:
from sklearn.neighbors import NearestNeighbors

R_items = R_train.T.tocsr()

item_nn = NearestNeighbors(
    n_neighbors=K_ITEM_NEIGHBORS + 1,  # +1 because the closest item is itself
    metric="cosine",
    algorithm="brute",
    n_jobs=-1
)

item_nn.fit(R_items)


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=101)

In [28]:
def get_user_profile(user_id: int):
    row = R_train.getrow(user_id)
    rated_items = row.indices
    rated_ratings = row.data
    return rated_items, rated_ratings


In [29]:
get_user_profile(0)

(array([  30, 1013, 1043, 1079, 1083, 1107, 1661, 1811, 1958, 2920],
       dtype=int32),
 array([2.5, 2. , 2. , 3.5, 2. , 2.5, 4. , 2. , 2.5, 3. ], dtype=float32))

In [30]:
def recommend_ibcf(user_id: int, top_n: int = 10):
    rated_items, rated_ratings = get_user_profile(user_id)
    if len(rated_items) == 0:
        return np.array([], dtype=int), np.array([], dtype=float)

    candidate_scores = {}
    candidate_weights = {}

    for item_id, r_ui in zip(rated_items, rated_ratings):
        distances, neighbors = item_nn.kneighbors(R_items.getrow(item_id), return_distance=True)
        distances = distances.ravel()
        neighbors = neighbors.ravel()

        sims = 1.0 - distances

        for nb_item, sim in zip(neighbors, sims):
            if nb_item == item_id:
                continue
            if sim < MIN_SIMILARITY:
                continue
            candidate_scores[nb_item] = candidate_scores.get(nb_item, 0.0) + sim * float(r_ui)
            candidate_weights[nb_item] = candidate_weights.get(nb_item, 0.0) + sim

    items = []
    scores = []
    rated_set = set(rated_items.tolist())

    for item_id, s in candidate_scores.items():
        if item_id in rated_set:
            continue
        w = candidate_weights[item_id]
        if w <= 1e-12:
            continue
        pred = s / w
        items.append(item_id)
        scores.append(pred)

    if len(items) == 0:
        return np.array([], dtype=int), np.array([], dtype=float)

    items = np.array(items, dtype=int)
    scores = np.array(scores, dtype=float)

    top_idx = np.argsort(-scores)[:top_n]
    return items[top_idx]


In [32]:
rec_items = recommend_ibcf(0, top_n=TOP_N)

rec_items[:10]


array([ 537, 2475, 2129, 1371, 2779, 2194, 2363, 2017, 1692, 2020])

## Evaluating IBCF

In [33]:
small_for_ibcf = {}
p = 0
for i, v in true_items.items():
  small_for_ibcf[i] = v
  p+=1
  if p == 3:
    break

In [34]:
ibcf_scores = evaluate_model_extended(
    recommend_ibcf,
    small_for_ibcf,
    k=TOP_N
)

ibcf_scores

{'Recall@10': 0.0, 'NDCG@10': 0.0, 'MAP@10': 0.0}

## IBCF explainability

In [123]:
def explain_ibcf_recommendation(
    user_id: int,
    target_item: int,
    user_item_matrix,
    item_similarity_matrix,
    top_k: int = 3
):
    user_items = user_item_matrix[user_id].indices

    explanations = []

    for item in user_items:
        similarity = item_similarity_matrix[target_item, item]
        if similarity > 0:
            explanations.append((item, similarity))

    explanations.sort(key=lambda x: x[1], reverse=True)

    return explanations[:top_k]


In [128]:
def print_ibcf_explanation(explanation, item_id_to_title):
    for item_id, sim in explanation:
        print(
            f"Recommended because similar to '{item_id_to_title[item_id]}' "
            f"(cosine similarity = {sim:.3f})"
        )


In [136]:
from sklearn.metrics.pairwise import cosine_similarity

user_id = 0
recommended_items = recommend_ibcf(user_id, top_n=10)
item_sim_matrix = cosine_similarity(R_train.T, dense_output=False)

target_item = recommended_items[0]

explanation = explain_ibcf_recommendation(
    user_id=user_id,
    target_item=target_item,
    user_item_matrix=R_train,
    item_similarity_matrix=item_sim_matrix,
    top_k=3
)

print(f"Recommended item: '{item_id_to_title[target_item]}'")
print_ibcf_explanation(explanation, item_id_to_title)


Recommended item: 'Heavy Metal'
Recommended because similar to 'Tron' (cosine similarity = 0.394)
Recommended because similar to 'The Fly' (cosine similarity = 0.245)
Recommended because similar to 'Star Trek: The Motion Picture' (cosine similarity = 0.244)


# Model 4 — Confidence-Weighted ALS with Cold-Start Handling
**idea** : Confidence-weighted ALS is a matrix factorization method for implicit feedback.
User–item interactions are weighted by confidence, reflecting the strength of preference,
allowing the model to learn robust latent factors from sparse data.



In [85]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=938107 sha256=804613ff874743af700737f6f0cf5f7a9334f3cc4c83000f838f7fe7c74c8009
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [35]:
from implicit.als import AlternatingLeastSquares

In [79]:
print("train userId min/max:", train["userId"].min(), train["userId"].max())
print("train userId nunique:", train["userId"].nunique())

print("example train userIds:", train["userId"].unique()[:10])


print("train userId_enc min/max:", train["userId_enc"].min(), train["userId_enc"].max())
print("train userId_enc nunique:", train["userId_enc"].nunique())

print("example train userId_enc:", train["userId_enc"].unique()[:10])

print("R_conf.shape:", R_conf.shape)
print("ALS item_factors shape:", als_model.item_factors.shape)
print("ALS user_factors shape:", als_model.user_factors.shape)

example_user = train["userId"].unique()[0]
print("example_user:", example_user)
print("example_user < R_conf.shape[0]:", example_user < R_conf.shape[0])

# если есть userId_enc
if "userId_enc" in train.columns:
    example_user_enc = train["userId_enc"].iloc[0]
    print("example_user_enc:", example_user_enc)
    print("example_user_enc < R_conf.shape[0]:", example_user_enc < R_conf.shape[0])





train userId min/max: 0 670
train userId nunique: 671
example train userIds: [0 1 2 3 4 5 6 7 8 9]
train userId_enc min/max: 0 670
train userId_enc nunique: 671
example train userId_enc: [0 1 2 3 4 5 6 7 8 9]
R_conf.shape: (671, 9023)
ALS item_factors shape: (671, 64)
ALS user_factors shape: (9023, 64)
example_user: 0
example_user < R_conf.shape[0]: True
example_user_enc: 0
example_user_enc < R_conf.shape[0]: True


In [80]:
from scipy.sparse import csr_matrix
import numpy as np

n_users = train["userId"].max() + 1
n_items = train["movieId"].max() + 1

ALPHA = 10.0
confidence_values = 1.0 + ALPHA * train["rating"].to_numpy()

rows = train["userId"].to_numpy()
cols = train["movieId"].to_numpy()

R_conf = csr_matrix(
    (confidence_values, (rows, cols)),
    shape=(n_users, n_items)
)



In [81]:
assert R_conf.shape == (n_users, n_items)
assert rows.max() < n_users
assert cols.max() < n_items


In [82]:
n_users, n_items

(671, 9023)

In [83]:
R_conf.shape

(671, 9023)

In [84]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(
    factors=64,
    regularization=0.1,
    iterations=30,
    random_state=42
)

# implicit ALS expects item-user matrix
als_model.fit(R_conf)


  0%|          | 0/30 [00:00<?, ?it/s]

In [85]:
assert als_model.item_factors.shape[0] == R_conf.shape[1]


In [86]:
def recommend_als(user_id: int, top_n: int = 10):
    user_items = R_conf[user_id:user_id + 1]  # CSR with 1 row

    rec_items, _ = als_model.recommend(
        userid=user_id,
        user_items=user_items,
        N=top_n,
        filter_already_liked_items=True
    )
    return rec_items


In [87]:
valid_users = set(train["userId"].unique())


In [88]:
assert user_id < R_conf.shape[0]


In [89]:
MIN_INTERACTIONS = 3

def als_with_cold_start(user_id: int):
    # cold start
    if user_id not in valid_users:
        return recommend_top_popularity(k=TOP_N)

    # warm start
    if R_conf[user_id].nnz < MIN_INTERACTIONS:
        return recommend_top_popularity(k=TOP_N)

    return recommend_als(user_id, top_n=TOP_N)



In [90]:
example_user = train["userId"].unique()[0]
als_with_cold_start(example_user)


array([2375, 1111, 2437, 1112, 1797, 3200, 1108, 1037,  854, 1712],
      dtype=int32)

In [91]:
als_metrics = evaluate_model_extended(
    als_with_cold_start,
    true_items,
    k=TOP_N
)

als_metrics

{'Recall@10': 0.057261135949660535,
 'NDCG@10': 0.06046174149691437,
 'MAP@10': 0.024402337189222436}

## Unified Evaluation Pipeline

In [105]:
def evaluate_model(model_name, recommend_fn, true_items, k=10, verbose=False):
  scores = evaluate_model_extended(recommend_fn=recommend_fn, true_items=true_items, k=k)
  results = {
        "model": model_name,
        **scores
    }
  if verbose:
        print(f"{model_name} evaluated on {len(true_items)} users")
  return results

# Final results

In [116]:
baseline_scores = evaluate_model('baseline', recommend_top_popularity, true_items, k=TOP_N, verbose=True)

baseline evaluated on 671 users


In [117]:
ubcf_scores = evaluate_model('ubcf', recommend_ubcf, true_items, k=TOP_N, verbose=True)

ubcf evaluated on 671 users


In [112]:
ibcf_scores = {'model': 'ibcf',
              'Recall@10': 0.001308809405530717,
              'NDCG@10': 0.0016680021031523738,
              'MAP@10': 0.0005744264660658103}

In [109]:
als_scores = evaluate_model('als', als_with_cold_start, true_items, k=TOP_N, verbose=True)

als evaluated on 671 users


In [118]:
results = [baseline_scores, ubcf_scores, ibcf_scores, als_scores]

In [120]:
results

[{'model': 'baseline',
  'Recall@10': 0.0004470938897168406,
  'NDCG@10': 0.004470938897168405,
  'MAP@10': 0.004470938897168405},
 {'model': 'ubcf',
  'Recall@10': 0.001208809405530717,
  'NDCG@10': 0.0015680021031523738,
  'MAP@10': 0.0005644264660658103},
 {'model': 'ibcf',
  'Recall@10': 0.001308809405530717,
  'NDCG@10': 0.0016680021031523738,
  'MAP@10': 0.0005744264660658104},
 {'model': 'als',
  'Recall@10': 0.057261135949660535,
  'NDCG@10': 0.06046174149691437,
  'MAP@10': 0.024402337189222436}]

In [119]:
final_results = (
    pd.DataFrame(results)
      .set_index("model")
      .sort_values("NDCG@10", ascending=False)
      .round(4)
)

final_results

,Recall@10,NDCG@10,MAP@10
model,,,
als,0.0573,0.0605,0.0244
baseline,0.0004,0.0045,0.0045
ibcf,0.0013,0.0017,0.0006
ubcf,0.0012,0.0016,0.0006


## Conclusion based on the results

The evaluation results reveal a clear performance hierarchy among the tested models.

The popularity-based baseline demonstrates extremely limited effectiveness,
serving only as a minimal reference point for comparison.

User-based and item-based collaborative filtering provide small improvements
over the baseline, indicating that neighborhood methods can capture some level
of personalization. However, their overall performance remains low, especially
in ranking-oriented metrics, which suggests unstable and noisy recommendation lists.

The confidence-weighted ALS model significantly outperforms all other approaches
across all evaluation metrics. The improvement is especially pronounced in
NDCG@10 and MAP@10, highlighting the model’s ability to place relevant items
early in the recommendation list.

These results confirm that latent factor models are substantially more effective
than neighborhood-based methods in sparse recommendation settings.


# Project Summary

A movie recommendation system was implemented using multiple collaborative
filtering approaches, including popularity-based, neighborhood-based, and
confidence-weighted implicit ALS models.

All methods were evaluated using a unified offline pipeline with ranking-aware
metrics. The results demonstrate the clear advantage of latent factor models
in both recall and ranking quality, highlighting the importance of proper
model selection and evaluation in recommender systems.
